# 基于 Chat Completions API 实现外部函数调用

2023年6月20日，OpenAI 官方在 Chat Completions API 原有的三种不同角色设定（System, Assistant, User）基础上，新增了 Function Calling 功能。

[详见OpenAI Blog](https://openai.com/blog/function-calling-and-other-api-updates)

## 概述

`functions` 是 Chat Completion API 中的可选参数，用于提供函数定义，其目的是使 GPT 模型能够生成符合所提供定义的函数参数。请注意，API 不会实际执行任何函数调用。

换句话说，我们在已有函数的前提下（该函数也可由 GPT 生成），将函数名称、功能以及参数告诉 GPT，之后 API 协助我们对用户的输入进行分析，比如在信息不全时，引导用户补充更多作为函数入参的信息。在应该调用对应函数的时候，告诉开发人员应该应该使用那一个函数，以及对应的入参分别是什么。

主要请求参数说明：

- **`messages` (array, Required)**

  A list of messages comprising the conversation so far. [Example Python code](https://cookbook.openai.com/examples/how_to_format_inputs_to_chatgpt_models).

  - **`function_call` (object, Optional)**
    
    The name and arguments of a function that should be called, as generated by the model.

    - **`name` (string, Required)**
            
      The name of the function to call.
      
    - **`arguments` (string, Required)**
            
      The arguments to call the function with, as generated by the model in JSON format. Note that the model does not always generate valid JSON, and may hallucinate parameters not defined by your function schema. Validate the arguments in your code before calling your function.


- **`functions` (array, Optional)**

  A list of functions the model may generate JSON inputs for.

  - **`name` (string, Required)**
            
    The name of the function to be called. Must be a-z, A-Z, 0-9, or contain underscores and dashes, with a maximum length of 64.
      
  - **`description` (string, Optional)**
            
    A description of what the function does, used by the model to choose when and how to call the function.

  - **`parameters` (object, Required)**
    
    The parameters the functions accepts, described as a JSON Schema object. See the [guide](https://platform.openai.com/docs/guides/gpt/function-calling) for examples, and the [JSON Schema reference](https://json-schema.org/understanding-json-schema/) for documentation about the format.
    
- **`function_call` (string or undefined, Optional)**

  Controls how the model responds to function calls. 
    - `none` means the model does not call a function, and responds to the end-user. 
    - `auto` means the model can pick between an end-user or calling a function. 
    - Specifying a particular function via `{"name": "my_function"}` forces the model to call that function. 

  `none` is the default when no functions are present. `auto` is the default if functions are present.


如果提供了 `functions` 参数，默认情况下，GPT 模型将决定在何时适当地使用其中一个函数。

可以通过将 `function_call` 参数设置为 `{"name": "<insert-function-name>"}` 来强制 API 使用指定函数。

同时，也支持通过将 `function_call` 参数设置为 `"none"` 来强制API不使用任何函数。

如果使用了某个函数，则响应中的输出将包含 `"finish_reason": "function_call"`，以及一个具有该函数名称和生成的函数参数的 `function_call` 对象。

## 环境配置

In [ ]:
%pip install scipy tenacity tiktoken termcolor openai requests

In [1]:
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
import os
import subprocess
from pprint import pprint

# 通过 subprocess 执行 shell 命令，获取 git 仓库的根目录
command = ['git', 'rev-parse', '--show-toplevel']
process = subprocess.Popen(command, stdout=subprocess.PIPE)
output, error = process.communicate()
git_root = output.decode().strip()

config_path = os.path.join(git_root, "config.json")
config = {}
with open(config_path,"r") as f:
    config = json.load(f)
openai.api_key = config["sk"]

GPT_MODEL = "gpt-3.5-turbo"

### 定义工具函数

定义一些用于调用聊天完成 API 的实用工具，并维护和跟踪对话状态。

In [2]:
# 使用了 retry 库，指定在请求失败时的重试策略。
# 这里设定的是指数等待（wait_random_exponential），时间间隔的最大值为40秒，并且最多重试3次（stop_after_attempt(3)）。
# 定义一个函数 chat_completion_request，主要用于发送 聊天补全 请求到 OpenAI 服务器
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=[], function_call='auto', model=GPT_MODEL):
    # 使用 openai 库来调用 chat 接口
    try:
        # 在使用 functions 参数时，需要确保 functions 入参一定是非空数组，不然会报错:
        #       None is not of type 'array' - 'functions'.
        #       [] is too short - 'functions'.
        # 在使用 function_call 参数时，需要确保正确使用了 functions 参数:
        #       Invalid value for 'function_call': 'function_call' is only allowed when 'functions' are specified.
        # 同时也要保障 function_call 是字典类型，或者是字符串类型且等于 'none' 'auto'

        def process_function_call(function_call):
            if type(function_call) == dict:
                return function_call
            elif type(function_call) == str and function_call in ['none', 'auto']:
                return function_call
            else:
                return 'auto'

        if functions is not None and isinstance(functions, list) and len(functions) > 0:
            response = openai.ChatCompletion.create(
                model=GPT_MODEL,
                messages=messages,
                functions=functions,
                function_call=process_function_call(function_call),
                max_tokens=2000,
            )
        else:
            response = openai.ChatCompletion.create(
                model=GPT_MODEL,
                messages=messages,
                max_tokens=2000,
            )
        # 返回服务器的响应
        return response

    # 如果发送请求或处理响应时出现异常，打印异常信息并返回
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [3]:
# 定义一个函数pretty_print_conversation，用于打印消息对话内容
def pretty_print_conversation(messages):
    # vscode 中 colored 函数不生效，改为直接使用转义字符来控制，语法为：
    #  \033[显示方式;前景色;背景色m要打印的文字\033[0m 
    #
    # \033：这是 ASCII 转义序列的起始符号，用来表示后面的代码是控制字符。
    # [0;31m：这是 ANSI 转义序列中设置颜色的代码。其中 [0 是样式代码，表示使用默认样式；31 是颜色代码，表示红色。
    # %s：这是一个占位符，表示要格式化输出的字符串。
    # \033[0m：这是 ANSI 转义序列中重置样式的代码，用来恢复到默认样式。

    # 为不同角色设置不同的颜色
    # role_to_color = {
    #     "system": "red",
    #     "user": "green",
    #     "assistant": "blue",
    #     "function": "magenta",
    # }

    # 遍历消息列表
    for message in messages:

        # 如果消息的角色是"system"，则用红色打印“content”
        if message["role"] == "system":
            # print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
            print("\033[0;31m%s\033[0m" % f"system: {message['content']}\n")

        # 如果消息的角色是"user"，则用绿色打印“content”
        elif message["role"] == "user":
            # print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
            print("\033[0;32m%s\033[0m" % f"user: {message['content']}\n")

        # 如果消息的角色是"assistant"，并且消息中包含"function_call"，则用蓝色打印"function_call"
        elif message["role"] == "assistant" and message.get("function_call"):
            # print(colored(f"assistant[function_call]: {message['function_call']}\n", role_to_color[message["role"]]))
            print("\033[0;34m%s\033[0m" % f"assistant[function_call]: {message['function_call']}\n")
        
        # 如果消息的角色是"assistant"，但是消息中不包含"function_call"，则用蓝色打印“content”
        elif message["role"] == "assistant" and not message.get("function_call"):
            # print(colored(f"assistant[content]: {message['content']}\n", role_to_color[message["role"]]))
            print("\033[0;34m%s\033[0m" % f"assistant[content]: {message['content']}\n")

        # 如果消息的角色是"function"，则用品红色打印“function”
        elif message["role"] == "function":
            # print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))
            print("\033[0;35m%s\033[0m" % f"assistant[content]: {message['content']}\n")

## 如何使用 functions 参数

这段代码定义了两个可以在程序中调用的函数，分别是获取当前天气和获取未来N天的天气预报。

每个函数(function)都有其名称、描述和需要的参数（包括参数的类型、描述等信息）。

### 定义函数

In [4]:
# 定义一个名为 functions 的列表，其中包含两个字典，这两个字典分别定义了两个功能的相关参数

functions = [
    # 第一个字典定义了一个名为 "get_current_weather" 的函数
    {
        "name": "get_current_weather",  # 函数的名称
        "description": "Get the current weather",  # 函数的描述
        "parameters": {  # 定义该函数需要的参数
            "type": "object",
            "properties": {  # 参数的属性
                "location": {  # 地点参数
                    "type": "string",  # 参数类型为字符串
                    "description": "The city and state, e.g. San Francisco, CA",  # 参数的描述
                },
                "format": {  # 温度单位参数
                    "type": "string",  # 参数类型为字符串
                    "enum": ["celsius", "fahrenheit"],  # 参数的取值范围
                    "description": "The temperature unit to use. Infer this from the users location.",  # 参数的描述
                },
            },
            "required": ["location", "format"],  # 该函数需要的必要参数
        },
    },
    # 第二个字典定义了一个名为 "get_n_day_weather_forecast" 的函数
    {
        "name": "get_n_day_weather_forecast",  # 函数的名称
        "description": "Get an N-day weather forecast",  # 函数的描述
        "parameters": {  # 定义该函数需要的参数
            "type": "object",
            "properties": {  # 参数的属性
                "location": {  # 地点参数
                    "type": "string",  # 参数类型为字符串
                    "description": "The city and state, e.g. San Francisco, CA",  # 参数的描述
                },
                "format": {  # 温度单位参数
                    "type": "string",  # 参数类型为字符串
                    "enum": ["celsius", "fahrenheit"],  # 参数的取值范围
                    "description": "The temperature unit to use. Infer this from the users location.",  # 参数的描述
                },
                "num_days": {  # 预测天数参数
                    "type": "integer",  # 参数类型为整数
                    "description": "The number of days to forecast",  # 参数的描述
                }
            },
            "required": ["location", "format", "num_days"]  # 该函数需要的必要参数
        },
    },
]

### 触发 `get_current_weather` 函数

这段代码首先定义了一个 `messages` 列表用来存储聊天的消息，然后向列表中添加了系统和用户的消息。

然后，它使用了之前定义的 `chat_completion_request` 函数发送一个请求，传入的参数包括消息列表和函数列表。

在接收到响应后，它从 JSON 响应中解析出 `assistant` 的消息，并将其添加到消息列表中。

最后，它打印出 GPT 模型回复的消息。

**（如果我们询问当前天气，GPT 模型会回复让你给出更准确的问题。）**

In [5]:
# 定义一个空列表messages，用于存储聊天的内容
messages = []

# 使用append方法向messages列表添加一条系统角色的消息
messages.append({
    "role": "system",  # 消息的角色是"system"
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."  # 消息的内容
})

# 向messages列表添加一条用户角色的消息
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "What's the weather like today"  # 用户询问今天的天气情况
})

try:
   # 使用定义的chat_completion_request函数发起一个请求，传入messages和functions作为参数
    chat_response = chat_completion_request(
        messages, functions=functions
    )

    # 解析返回的JSON数据，获取assistant的回复消息
    assistant_message = chat_response.choices[0].message

    # 将assistant的回复消息添加到messages列表中
    messages.append(json.loads(str(assistant_message)))

    pretty_print_conversation(messages)

# 如果发送请求或处理响应时出现异常，打印异常信息并返回
except Exception as e:
    print(f"Exception: {e}")

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant[content]: Sure, could you please provide me with your current location?



#### 补充 `location` 参数

下面这段代码先向 `messages` 列表中添加了用户的位置信息。

然后再次使用了 `chat_completion_request` 函数发起请求，只是这次传入的消息列表已经包括了用户的新消息。

在获取到响应后，它同样从 `JSON` 响应中解析出 `assistant` 的消息，并将其添加到消息列表中。

最后，打印出 `assistant` 的新的回复消息。

In [6]:
# 向messages列表添加一条用户角色的消息，用户告知他们在上海
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "I'm in Shanghai, China."  # 用户的消息内容
})

# 再次使用定义的chat_completion_request函数发起一个请求，传入更新后的messages和functions作为参数
chat_response = chat_completion_request(
    messages, functions=functions
)

# 解析返回的JSON数据，获取assistant的新的回复消息
assistant_message = chat_response.choices[0].message

# 将assistant的新的回复消息添加到messages列表中
messages.append(json.loads(str(assistant_message)))

# 打印assistant的回复消息
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant[content]: Sure, could you please provide me with your current location?

user: I'm in Shanghai, China.

assistant[function_call]: {'arguments': '{\n  "location": "Shanghai, China",\n  "format": "celsius"\n}', 'name': 'get_current_weather'}



### 触发 `get_curreget_n_day_weather_forecastnt_weather` 函数

这段代码的逻辑大体与上一段代码相同，区别在于这次用户的询问中涉及到未来若干天（x天）的天气预报。

在获取到回复后，它同样从 JSON 响应中解析出 `assistant` 的消息，并将其添加到消息列表中。

然后打印出 `assistant` 的回复消息。

**（通过不同的prompt方式，我们可以让它针对我们告诉它的其他功能。）**

In [7]:
# 初始化一个空的messages列表
messages = []

# 向messages列表添加一条系统角色的消息，要求不做关于函数参数值的假设，如果用户的请求模糊，应该寻求澄清
messages.append({
    "role": "system",  # 消息的角色是"system"
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
})

# 向messages列表添加一条用户角色的消息，用户询问在未来x天内苏格兰格拉斯哥的天气情况
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "what is the weather going to be like in Shanghai, China over the next x days"
})

# 使用定义的chat_completion_request函数发起一个请求，传入messages和functions作为参数
chat_response = chat_completion_request(
    messages, functions=functions
)

# 解析返回的JSON数据，获取assistant的新的回复消息
assistant_message = chat_response.choices[0].message

# 将assistant的新的回复消息添加到messages列表中
messages.append(json.loads(str(assistant_message)))

# 打印assistant的回复消息
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: what is the weather going to be like in Shanghai, China over the next x days

assistant[content]: Sure, I can help you with that. Please let me know the number of days you would like to forecast for Shanghai, China.



#### 补充 `num_days` 参数

**(GPT 模型再次要求我们澄清，因为它还没有足够的信息。在这种情况下，它已经知道预测的位置，但需要知道需要多少天的预测。)**

这段代码的主要目标是将用户指定的天数（5天）添加到消息列表中，然后再次调用 chat_completion_request 函数发起一个请求。

返回的响应中包含了助手对用户的回复，即未来5天的天气预报。

这个预报是基于用户指定的地点（上海）和天数（5天）生成的。

在代码的最后，它解析出返回的 JSON 响应中的第一个选项，这就是助手的回复消息。

In [8]:
# 向messages列表添加一条用户角色的消息，用户指定接下来的天数为5天
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "5 days"
})

# 使用定义的chat_completion_request函数发起一个请求，传入messages和functions作为参数
chat_response = chat_completion_request(
    messages, functions=functions
)

# 解析返回的JSON数据，获取assistant的新的回复消息
assistant_message = chat_response.choices[0].message

# 将assistant的新的回复消息添加到messages列表中
messages.append(json.loads(str(assistant_message)))

# 打印assistant的回复消息
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: what is the weather going to be like in Shanghai, China over the next x days

assistant[content]: Sure, I can help you with that. Please let me know the number of days you would like to forecast for Shanghai, China.

user: 5 days

assistant[function_call]: {'arguments': '{\n  "location": "Shanghai, China",\n  "format": "celsius",\n  "num_days": 5\n}', 'name': 'get_n_day_weather_forecast'}



### 强制使用指定函数

我们可以通过使用 `function_call` 参数来强制GPT 模型使用指定函数，例如 `get_n_day_weather_forecast`。

通过这种方式，可以让 GPT 模型学习如何使用该函数。

#### 不指定任何函数

下面这段代码演示了在不强制使用特定函数（`get_n_day_weather_forecast`）的情况下，GPT 模型可能会选择不同的方式来回应用户的请求。对于给定的用户请求"Give me a weather report for San Diego, USA."，GPT 模型可能不会调用`get_n_day_weather_forecast`函数。

In [9]:
# 如果我们不强制GPT 模型使用 get_n_day_weather_forecast，它可能不会使用
messages = []  # 创建一个空的消息列表

# 添加系统角色的消息
messages.append({
    "role": "system",  # 角色为系统
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
})

# 添加用户角色的消息
messages.append({
    "role": "user",  # 角色为用户
    "content": "Give me a weather report for San Diego, USA."
})

# 使用定义的chat_completion_request函数发起一个请求，传入messages和functions作为参数
chat_response = chat_completion_request(
    messages, functions=functions
)

# 解析返回的JSON数据，获取assistant的新的回复消息
assistant_message = chat_response.choices[0].message

# 将assistant的新的回复消息添加到messages列表中
messages.append(json.loads(str(assistant_message)))

# 打印assistant的回复消息
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: Give me a weather report for San Diego, USA.

assistant[function_call]: {'arguments': '{\n  "location": "San Diego, USA",\n  "format": "celsius"\n}', 'name': 'get_current_weather'}



#### 强制指定 `get_n_day_weather_forecast` 函数

通过对比可以看出，在未强制指定时，会判断应该使用 `get_current_weather` 函数。

而当强制指定后，则只会使用 `get_n_day_weather_forecast` 函数，且会根据语义信息，设定参数的取值，如在此示例中，将 n_day 设置为 1 天。

In [10]:
# 在这个代码单元中，我们强制GPT 模型使用get_n_day_weather_forecast函数
messages = []  # 创建一个空的消息列表

# 添加系统角色的消息
messages.append({
    "role": "system",  # 角色为系统
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
})

# 添加用户角色的消息
messages.append({
    "role": "user",  # 角色为用户
    "content": "Give me a weather report for San Diego, USA."
})

# 使用定义的chat_completion_request函数发起一个请求，传入messages、functions以及特定的function_call作为参数
chat_response = chat_completion_request(
    messages, functions=functions, function_call={"name": "get_n_day_weather_forecast"}
)

# 解析返回的JSON数据，获取assistant的新的回复消息
assistant_message = chat_response.choices[0].message

# 将assistant的新的回复消息添加到messages列表中
messages.append(json.loads(str(assistant_message)))

# 打印assistant的回复消息
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: Give me a weather report for San Diego, USA.

assistant[function_call]: {'arguments': '{\n  "location": "San Diego, USA",\n  "format": "celsius",\n  "num_days": 1\n}', 'name': 'get_n_day_weather_forecast'}



#### 强制不使用函数

然后，我们创建另一个消息列表，并添加系统和用户的消息。这次用户请求的是加拿大多伦多当前的天气（使用摄氏度）。

随后，代码再次调用 `chat_completion_request` 函数，

但这次在 `function_call` 参数中明确指定了"none"，表示 GPT 模型在处理此请求时不能调用任何函数。

最后，代码解析返回的 JSON 响应，获取第一个选项的消息，即 GPT 模型的回应。

从解析结果来看，GPT 仍然学习到了 functions 中的内容，会将用户的描述生成参数信息，但是不会去真正调用函数。

In [11]:
# 创建另一个空的消息列表
messages = []

# 添加系统角色的消息
messages.append({
    "role": "system",  # 角色为系统
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
})

# 添加用户角色的消息
messages.append({
    "role": "user",  # 角色为用户
    "content": "Give me the current weather (use Celcius) for Toronto, Canada."
})

# 使用定义的chat_completion_request函数发起一个请求，传入messages、functions和function_call作为参数
chat_response = chat_completion_request(
    messages, functions=functions, function_call="none"
)

# 解析返回的JSON数据，获取assistant的新的回复消息
assistant_message = chat_response.choices[0].message

# 将assistant的新的回复消息添加到messages列表中
messages.append(json.loads(str(assistant_message)))

# 打印assistant的回复消息
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: Give me the current weather (use Celcius) for Toronto, Canada.

assistant[content]: {
  "location": "Toronto, Canada",
  "format": "celsius"
}



### 实现城市天气查询功能

天气查询功能通过[和风天气](https://dev.qweather.com/)实现，支持通过 Api 获取指定城市的天气信息，可免费注册使用。

在查询城市时，Api 提供了 `adm`(城市的上级行政区划)、`range`(搜索范围，可设定只在某个国家或地区范围内进行搜索，国家和地区名称需使用[ISO 3166 所定义的国家代码](https://dev.qweather.com/docs/resource/glossary/#iso-3166)) 参数来更精确地定位城市，但是对于入参有较大限制，出于演示考虑，本次仅使用城市本身的名称作为搜索项。

In [12]:
qweather_sk = config["qweather_sk"]

#### 添加天气查询函数实现

In [13]:
def city_id_search(city, number=1):
    url = 'https://geoapi.qweather.com/v2/city/lookup'
    params = {
        'location': city,
        'key': qweather_sk,
        'number': number
    }

    response = requests.get(url, params=params)
    data = response.json()
    
    res = {}
    for location in data['location']:
        res[location['id']] = f"{location['country']}、{location['adm1']}、{location['adm2']}、{location['name']}"

    return res

def get_current_weather(location, format):
    # 出于演示以及 Api 入参限制考虑，仅查询城市本身
    city = location.split(",")[0]
    
    city_dict = city_id_search(city, number=5)
    res = "\n当前查询结果为：\n"
    url = 'https://devapi.qweather.com/v7/weather/now'
    for city_id in list(city_dict.keys()):
        params = {
            'location': city_id,
            'key': qweather_sk
        }
        response = requests.get(url, params=params).json()
        res += f"\t地点：{city_dict[city_id]}，时间：{response['now']['obsTime']}，温度：{response['now']['temp']} {format}\n"
    return res


def get_n_day_weather_forecast(location, format, num_days):
    # 出于演示以及 Api 入参限制考虑，仅查询城市本身
    city = location.split(",")[0]
    
    city_dict = city_id_search(city, number=5)
    res = f"\n{num_days}天后，天气结果为：\n"
    url = 'https://devapi.qweather.com/v7/weather/7d'
    for city_id in list(city_dict.keys()):
        params = {
            'location': city_id,
            'key': qweather_sk
        }
        response = requests.get(url, params=params).json()
        daily = response['daily'][num_days]

        res += f"\t地点：{city_dict[city_id]}，时间：{daily['fxDate']}，最高温度：{daily['tempMax']} {format}，最低温度：{daily['tempMin']} {format}\n"
    return res

#### 查询北京当前天气

In [14]:
# 定义一个空列表messages，用于存储聊天的内容
messages = []

# 使用append方法向messages列表添加一条系统角色的消息
messages.append({
    "role": "system",  # 消息的角色是"system"
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."  # 消息的内容
})

# 向messages列表添加一条用户角色的消息
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "北京市现在的天气怎么样？"  # 用户询问今天的天气情况
})

try:
   # 使用定义的chat_completion_request函数发起一个请求，传入messages和functions作为参数
    chat_response = chat_completion_request(
        messages, functions=functions
    )

    # 解析返回的JSON数据，获取assistant的回复消息
    assistant_message = chat_response.choices[0].message

    # 将assistant的回复消息添加到messages列表中
    messages.append(json.loads(str(assistant_message)))

    # 如果助手的消息中有功能调用
    if assistant_message.get("function_call"):
        if assistant_message["function_call"]["name"] == 'get_current_weather':
            arguments = json.loads(assistant_message["function_call"]["arguments"])
            # 使用 get_current_weather 函数执行功能调用，并获取结果
            results = get_current_weather(location=arguments["location"], format=arguments["format"])
            # 将功能的结果作为一个功能角色的消息添加到消息列表中
            messages.append({"role": "function", "content": results, "name": assistant_message["function_call"]["name"]})
        
    pretty_print_conversation(messages)

# 如果发送请求或处理响应时出现异常，打印异常信息并返回
except Exception as e:
    print(f"Exception: {e}")

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: 北京市现在的天气怎么样？

assistant[function_call]: {'arguments': '{\n  "location": "北京市",\n  "format": "celsius"\n}', 'name': 'get_current_weather'}

assistant[content]: 
当前查询结果为：
	地点：中国、北京市、北京、北京，时间：2023-10-02T20:56+08:00，温度：17 celsius
	地点：中国、北京市、北京、海淀，时间：2023-10-02T20:58+08:00，温度：16 celsius
	地点：中国、北京市、北京、朝阳，时间：2023-10-02T20:58+08:00，温度：18 celsius
	地点：中国、北京市、北京、昌平，时间：2023-10-02T20:56+08:00，温度：16 celsius
	地点：中国、北京市、北京、房山，时间：2023-10-02T20:58+08:00，温度：17 celsius




#### 查询北京 3 天后的天气

In [15]:
# 定义一个空列表messages，用于存储聊天的内容
messages = []

# 使用append方法向messages列表添加一条系统角色的消息
messages.append({
    "role": "system",  # 消息的角色是"system"
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."  # 消息的内容
})

# 向messages列表添加一条用户角色的消息
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "北京市三天后的天气怎么样？"  # 用户询问今天的天气情况
})

try:
   # 使用定义的chat_completion_request函数发起一个请求，传入messages和functions作为参数
    chat_response = chat_completion_request(
        messages, functions=functions
    )

    # 解析返回的JSON数据，获取assistant的回复消息
    assistant_message = chat_response.choices[0].message

    # 将assistant的回复消息添加到messages列表中
    messages.append(json.loads(str(assistant_message)))

    # 如果助手的消息中有功能调用
    if assistant_message.get("function_call"):
        if assistant_message["function_call"]["name"] == 'get_current_weather':
            arguments = json.loads(assistant_message["function_call"]["arguments"])
            # 使用 get_current_weather 函数执行功能调用，并获取结果
            results = get_current_weather(location=arguments["location"], format=arguments["format"])
            # 将功能的结果作为一个功能角色的消息添加到消息列表中
            messages.append({"role": "function", "content": results, "name": assistant_message["function_call"]["name"]})
        if assistant_message["function_call"]["name"] == 'get_n_day_weather_forecast':
            arguments = json.loads(assistant_message["function_call"]["arguments"])
            if arguments["num_days"] <= 7:
                # 使用 get_current_weather 函数执行功能调用，并获取结果
                results = get_n_day_weather_forecast(location=arguments["location"], format=arguments["format"], num_days=arguments["num_days"])
                # 将功能的结果作为一个功能角色的消息添加到消息列表中
                messages.append({"role": "function", "content": results, "name": assistant_message["function_call"]["name"]})
            else:
                print("日期超过限制，仅支持7天内天气预测")

    pretty_print_conversation(messages)

# 如果发送请求或处理响应时出现异常，打印异常信息并返回
except Exception as e:
    print(f"Exception: {e}")

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: 北京市三天后的天气怎么样？

assistant[function_call]: {'arguments': '{\n  "location": "北京市",\n  "format": "celsius",\n  "num_days": 3\n}', 'name': 'get_n_day_weather_forecast'}

assistant[content]: 
3天后，天气结果为：
	地点：中国、北京市、北京、北京，时间：2023-10-05，最高温度：23 celsius，最低温度：12 celsius
	地点：中国、北京市、北京、海淀，时间：2023-10-05，最高温度：24 celsius，最低温度：13 celsius
	地点：中国、北京市、北京、朝阳，时间：2023-10-05，最高温度：24 celsius，最低温度：13 celsius
	地点：中国、北京市、北京、昌平，时间：2023-10-05，最高温度：23 celsius，最低温度：9 celsius
	地点：中国、北京市、北京、房山，时间：2023-10-05，最高温度：24 celsius，最低温度：12 celsius




## 执行 GPT 模型生成的函数

接着，我们将演示如何执行输入为 GPT 模型生成的函数，并利用这一点来实现一个可以帮助我们回答关于数据库的问题的代理。

为了简单起见，我们将使用[Chinook样本数据库](https://www.sqlitetutorial.net/sqlite-sample-database/)。

*注意：* 在生产环境中，SQL生成可能存在较高风险，因为GPT 模型在生成正确的SQL方面并不完全可靠。

### 连接数据库

In [16]:
import os
import subprocess
import sqlite3

# 通过 subprocess 执行 shell 命令，获取 git 仓库的根目录
command = ['git', 'rev-parse', '--show-toplevel']
process = subprocess.Popen(command, stdout=subprocess.PIPE)
output, error = process.communicate()
git_root = output.decode().strip()

chinook_datapath = os.path.join(git_root, "openai-api", "data", "chinook.db")

conn = sqlite3.connect(chinook_datapath)
print("Opened database successfully")

Opened database successfully


### 定义一个执行 SQL 查询的函数

定义一些有用的实用函数来从 SQLite 数据库中提取数据。

#### 定义信息获取函数

定义三个函数 `get_table_names`、`get_column_names` 和 `get_database_info`，用于从数据库连接对象中获取数据库的表名、表的列名以及整体数据库的信息。

In [17]:
def get_table_names(conn):
    """返回一个包含所有表名的列表"""
    table_names = []  # 创建一个空的表名列表
    # 执行SQL查询，获取数据库中所有表的名字
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    # 遍历查询结果，并将每个表名添加到列表中
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names  # 返回表名列表


def get_column_names(conn, table_name):
    """返回一个给定表的所有列名的列表"""
    column_names = []  # 创建一个空的列名列表
    # 执行SQL查询，获取表的所有列的信息
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    # 遍历查询结果，并将每个列名添加到列表中
    for col in columns:
        column_names.append(col[1])
    return column_names  # 返回列名列表


def get_database_info(conn):
    """返回一个字典列表，每个字典包含一个表的名字和列信息"""
    table_dicts = []  # 创建一个空的字典列表
    # 遍历数据库中的所有表
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)  # 获取当前表的所有列名
        # 将表名和列名信息作为一个字典添加到列表中
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts  # 返回字典列表

#### 数据信息转为字典

In [18]:
# 获取数据库信息，并存储为字典列表
database_schema_dict = get_database_info(conn)

# 将数据库信息转换为字符串格式，方便后续使用
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

#### 构造 SQL 查询函数

定义一个函数 `ask_database`，目标是让 GPT 模型帮我们构造一个完整的 SQL 查询。

In [19]:
# 定义一个功能列表，其中包含一个功能字典，该字典定义了一个名为"ask_database"的功能，用于回答用户关于音乐的问题
functions = [
    {
        "name": "ask_database",
        "description": "Use this function to answer user questions about music. Output should be a fully formed SQL query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            SQL query extracting info to answer the user's question.
                            SQL should be written using this database schema:
                            {database_schema_string}
                            The query should be returned in plain text, not in JSON.
                            """,
                }
            },
            "required": ["query"],
        },
    }
]

### 执行 SQL 查询

首先，定义两个函数`ask_database`和`execute_function_call`
- 前者用于实际执行 SQL 查询并返回结果
- 后者用于根据消息中的功能调用信息来执行相应的功能并获取结果

然后，创建一个消息列表，并向其中添加了一个系统消息和一个用户消息。系统消息的内容是指示对话的目标，用户消息的内容是用户的问题。

接着，使用`chat_completion_request`函数发出聊天请求并获取响应，然后从响应中提取出助手的消息并添加到消息列表中。

如果助手的消息中包含功能调用，那么就使用`execute_function_call`函数执行这个功能调用并获取结果，然后将结果作为一个功能消息添加到消息列表中。

最后，使用`pretty_print_conversation`函数打印出整个对话。

In [20]:
def ask_database(conn, query):
    """使用 query 来查询 SQLite 数据库的函数。"""
    try:
        results = str(conn.execute(query).fetchall())  # 执行查询，并将结果转换为字符串
    except Exception as e:  # 如果查询失败，捕获异常并返回错误信息
        results = f"query failed with error: {e}"
    return results  # 返回查询结果


def execute_function_call(message):
    """执行函数调用"""
    # 判断功能调用的名称是否为 "ask_database"
    if message["function_call"]["name"] == "ask_database":
        # 如果是，则获取功能调用的参数，这里是 SQL 查询
        query = json.loads(message["function_call"]["arguments"])["query"]
        # 使用 ask_database 函数执行查询，并获取结果
        results = ask_database(conn, query)
    else:
        # 如果功能调用的名称不是 "ask_database"，则返回错误信息
        results = f"Error: function {message['function_call']['name']} does not exist"
    return results  # 返回结果

In [21]:
# 创建一个空的消息列表
messages = []

# 向消息列表中添加一个系统角色的消息，内容是 "Answer user questions by generating SQL queries against the Chinook Music Database."
messages.append({"role": "system", "content": "Answer user questions by generating SQL queries against the Chinook Music Database."})

# 向消息列表中添加一个用户角色的消息，内容是 "Hi, who are the top 5 artists by number of tracks?"
messages.append({"role": "user", "content": "Hi, who are the top 5 artists by number of tracks?"})

# 使用 chat_completion_request 函数获取聊天响应
chat_response = chat_completion_request(messages, functions)

# 从聊天响应中获取assistant的消息
assistant_message = chat_response.choices[0].message

# 将assistant的新的回复消息添加到messages列表中
messages.append(json.loads(str(assistant_message)))

# 如果助手的消息中有功能调用
if assistant_message.get("function_call"):
    # 使用 execute_function_call 函数执行功能调用，并获取结果
    results = execute_function_call(assistant_message)
    # 将功能的结果作为一个功能角色的消息添加到消息列表中
    messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})

# 使用 pretty_print_conversation 函数打印对话
pretty_print_conversation(messages)

system: Answer user questions by generating SQL queries against the Chinook Music Database.

user: Hi, who are the top 5 artists by number of tracks?

assistant[function_call]: {'arguments': '{\n  "query": "SELECT artists.Name, COUNT(tracks.TrackId) AS TrackCount FROM artists JOIN albums ON artists.ArtistId = albums.ArtistId JOIN tracks ON albums.AlbumId = tracks.AlbumId GROUP BY artists.Name ORDER BY TrackCount DESC LIMIT 5;"\n}', 'name': 'ask_database'}

assistant[content]: [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]



In [22]:
# 向消息列表中添加一个用户的问题，内容是 "What is the name of the album with the most tracks?"
messages.append({"role": "user", "content": "What is the name of the album with the most tracks?"})

# 使用 chat_completion_request 函数获取聊天响应
chat_response = chat_completion_request(messages, functions)

# 从聊天响应中获取assistant的消息
assistant_message = chat_response.choices[0].message

# 将assistant的新的回复消息添加到messages列表中
messages.append(json.loads(str(assistant_message)))

# 如果助手的消息中有功能调用
if assistant_message.get("function_call"):
    # 使用 execute_function_call 函数执行功能调用，并获取结果
    results = execute_function_call(assistant_message)
    # 将功能的结果作为一个功能角色的消息添加到消息列表中
    messages.append({"role": "function", "content": results, "name": assistant_message["function_call"]["name"]})

# 使用 pretty_print_conversation 函数打印对话
pretty_print_conversation(messages)

system: Answer user questions by generating SQL queries against the Chinook Music Database.

user: Hi, who are the top 5 artists by number of tracks?

assistant[function_call]: {'arguments': '{\n  "query": "SELECT artists.Name, COUNT(tracks.TrackId) AS TrackCount FROM artists JOIN albums ON artists.ArtistId = albums.ArtistId JOIN tracks ON albums.AlbumId = tracks.AlbumId GROUP BY artists.Name ORDER BY TrackCount DESC LIMIT 5;"\n}', 'name': 'ask_database'}

assistant[content]: [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]

user: What is the name of the album with the most tracks?

assistant[function_call]: {'arguments': '{\n  "query": "SELECT albums.Title, COUNT(tracks.TrackId) AS TrackCount FROM albums JOIN tracks ON albums.AlbumId = tracks.AlbumId GROUP BY albums.Title ORDER BY TrackCount DESC LIMIT 1;"\n}', 'name': 'ask_database'}

assistant[content]: [('Greatest Hits', 57)]

